# AlphaEarth Change Detection

This notebook demonstrates how to detect changes over time using AlphaEarth satellite embeddings.

## How Change Detection Works

AlphaEarth embeddings capture semantic information about Earth's surface. By comparing embeddings from different years, we can identify areas that have changed:

- **Dot Product Similarity**: Measures how similar two embedding vectors are
- **High similarity (~1.0)**: Areas that remain stable
- **Low similarity (~0.0)**: Areas that have changed significantly

---

**Attribution:** The AlphaEarth Foundations Satellite Embedding dataset is produced by Google and Google DeepMind.

## Setup

In [ ]:
import ee
import leafmap.maplibregl as leafmap

In [ ]:
# Initialize Earth Engine (configure with your project)
# ee.Authenticate()
# ee.Initialize(project="YOUR_PROJECT_ID")

## Loading AlphaEarth Embeddings Programmatically

Instead of using the GUI, let's load embeddings directly from the Earth Engine dataset.

In [ ]:
# AlphaEarth dataset
dataset = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")

# Configuration
year1 = 2017
year2 = 2024

# Load images for both years
image1 = dataset.filter(ee.Filter.eq("year", year1)).first()
image2 = dataset.filter(ee.Filter.eq("year", year2)).first()

print(f"Loaded embeddings for {year1} and {year2}")

## Comparing Two Years Visually

First, let's visualize the embeddings from both years side by side.

In [ ]:
# Location: San Francisco Bay Area
center_lat = 37.8
center_lon = -122.4
zoom = 12

# Create map
m = leafmap.Map(center=[center_lat, center_lon], zoom=zoom)
m.add_basemap("USGS.Imagery")

# Visualization parameters
vis_params = {
    "bands": ["A01", "A16", "A09"],
    "min": -0.3,
    "max": 0.3,
}

# Add both years as layers
m.add_ee_layer(image1, vis_params, f"AlphaEarth {year1}")
m.add_ee_layer(image2, vis_params, f"AlphaEarth {year2}")

m.add_layer_control()
m

## Calculating Change Using Dot Product

The dot product between two normalized embedding vectors measures their similarity:
- Result close to 1: Vectors are similar (stable)
- Result close to 0: Vectors are different (changed)

In [ ]:
def calculate_similarity(img1, img2):
    """
    Calculate dot product similarity between two AlphaEarth images.
    
    Returns values from 0 (different) to 1 (identical).
    """
    # Get all 64 embedding band names
    band_names = [f"A{str(i).zfill(2)}" for i in range(1, 65)]
    
    # Select embedding bands
    emb1 = img1.select(band_names)
    emb2 = img2.select(band_names)
    
    # Dot product: element-wise multiply then sum across bands
    dot_product = emb1.multiply(emb2).reduce(ee.Reducer.sum())
    
    # Clamp to valid range
    similarity = dot_product.clamp(0, 1)
    
    return similarity.rename("similarity")

In [ ]:
# Calculate similarity between years
similarity = calculate_similarity(image1, image2)
print("Similarity calculation complete")

## Visualizing Change Detection Results

In [ ]:
# Create map with change detection
m = leafmap.Map(center=[center_lat, center_lon], zoom=zoom)
m.add_basemap("USGS.Imagery")

# Add embedding layers
m.add_ee_layer(image1, vis_params, f"Embeddings {year1}")
m.add_ee_layer(image2, vis_params, f"Embeddings {year2}")

# Change detection visualization
# White = stable (high similarity)
# Black = changed (low similarity)
change_vis = {
    "min": 0,
    "max": 1,
    "palette": ["white", "black"],
}

m.add_ee_layer(similarity, change_vis, "Change Detection")
m.add_layer_control()
m

## Interpreting Results

In the change detection layer:

- **White areas**: High similarity - stable, unchanged
- **Black areas**: Low similarity - significant change detected
- **Gray areas**: Partial change or gradual transition

### Common Types of Changes Detected:

1. **Urban Development**: New buildings, roads, infrastructure
2. **Agricultural Changes**: Crop rotation, land conversion
3. **Vegetation Changes**: Deforestation, reforestation, seasonal variations
4. **Water Level Changes**: Flooding, drought, reservoir levels
5. **Natural Disasters**: Fire scars, landslides, erosion

## Example Locations to Explore

Try changing the `center_lat`, `center_lon`, and re-running the cells above for these interesting locations:

### Urban Growth
- **Dubai, UAE**: lat=25.2, lon=55.3 (massive urban expansion)
- **Phoenix, AZ**: lat=33.4, lon=-112.0 (suburban sprawl)

### Agricultural Changes
- **Central California**: lat=36.5, lon=-120.5 (irrigation patterns)
- **Amazon Region**: lat=-3.0, lon=-60.0 (deforestation)

### Environmental Changes
- **Aral Sea**: lat=45.0, lon=59.0 (dramatic water loss)
- **Great Salt Lake**: lat=41.0, lon=-112.5 (shrinking lake)

## Next Steps

- See **03_advanced_analysis.ipynb** for custom band combinations and batch processing
- Explore Google's official tutorials for machine learning applications:
  [Satellite Embedding Tutorials](https://developers.google.com/earth-engine/tutorials/community/satellite-embedding-01-introduction)